## 데이터 불러오기

In [66]:
from autogluon.tabular import TabularDataset, TabularPredictor
import dask.dataframe as dd
import vaex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spst
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [67]:
card_data = '../data/gyeonggi_card_2023.csv'
flowpop_data = '../data/flowpop.csv'
commercial_area_ndc_data = '../data/commercial_area_ndc.csv'
card_sample = '../data/gm_card.csv'

In [68]:
# card_df = dd.read_csv(card_sample,encoding='utf-8')

In [98]:
card_df = dd.read_csv(card_data,encoding='utf-8')
flowpop_df = dd.read_csv(flowpop_data,encoding='utf-8')
comm_df = dd.read_csv(commercial_area_ndc_data,encoding='utf-8')

## 데이터 전처리

#### 카드 소비 데이터

In [99]:
card_df.head()

,ta_ymd,cty_rgn_no,admi_cty_no,card_tpbuz_cd,card_tpbuz_nm_1,card_tpbuz_nm_2,hour,sex,age,day,amt,cnt
0,20230101,41210,41210510,D05,소매/유통,선물/완구,3,M,5,7,57576,2
1,20230101,41210,41210510,D11,소매/유통,종합소매점,3,F,7,7,8523,2
2,20230101,41210,41210510,D11,소매/유통,종합소매점,7,F,7,7,8023,2
3,20230101,41210,41210510,F02,생활서비스,미용서비스,3,M,4,7,28532,2
4,20230101,41210,41210510,F02,생활서비스,미용서비스,4,F,7,7,74074,2


In [ ]:
#card_df = card_df.compute()

In [ ]:
#card_df.dtypes

In [28]:
flowpop_df.head()

,ETL_M,ADMI_CD,CTY_NM,ADMI_NM,TIME_CD,M_CNT,F_CNT,10_CNT,20_CNT,30_CNT,40_CNT,50_CNT,60_CNT,70_CNT,TOTAL_CNT,TOTAL_POPULATION
0,202301,41210510,광명시,광명1동,0,135.01,99.40,10.78,8.93,33.53,41.47,89.62,40.75,9.33,234.41,234.41
1,202301,41210510,광명시,광명1동,1,142.90,97.38,11.53,6.52,41.38,41.75,87.49,40.88,10.73,240.28,240.28
2,202301,41210510,광명시,광명1동,2,150.74,103.40,13.29,6.45,42.03,47.87,87.45,46.32,10.73,254.14,254.14
3,202301,41210510,광명시,광명1동,3,161.26,110.05,14.88,6.43,36.72,52.90,99.85,49.61,10.92,271.31,271.31
4,202301,41210510,광명시,광명1동,4,187.24,123.51,15.40,8.66,45.70,64.74,105.75,58.80,11.70,310.75,310.75


In [ ]:
flowpop_df.dtypes

In [33]:
comm_df.head()

,행정시명,행정동명,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명,기준분기
0,시흥시,대야동,84.9,66.4,HH,정체,2
1,시흥시,신천동,93.6,54.1,HL,상권축소,2
2,시흥시,신현동,88.7,102.4,HH,정체,2
3,시흥시,은행동,50.9,41.9,LL,다이나믹,2
4,시흥시,매화동,92.2,48.1,HL,상권축소,2


### 사용하지 않는 속성 제거

In [28]:
#flowpop_df.drop(['')],axis=1, inplace=True)

In [ ]:
# 필요한 컬럼만 선택하여 병합
# merged_df = pd.merge(card_df[['ta_ymd', 'admi_cty_no', 'amt']], 
#                      flowpop_df[['TIME_CD', 'ADMI_CD', 'M_10_CNT', 'F_10_CNT']], 
#                      how='inner', 
#                      left_on=['ta_ymd', 'admi_cty_no'], 
#                      right_on=['TIME_CD', 'ADMI_CD'])

# 두 DataFrame을 날짜와 admi_cd를 기준으로 병합
#merged_df = dd.merge(card_df, flowpop_df, how='inner', left_on=['ta_ymd', 'admi_cty_no'], right_on=['ETL_YMD', 'ADMI_CD'])

# 중복 컬럼 제거
# merged_df.drop(columns=['TIME_CD', 'ADMI_CD'], inplace=True)

In [ ]:
#merged_df.head()

## 카드 소비 데이터만 학습

### 범주형 데이터 변환

In [44]:
# enc_classes = {} 
# def encoding_label(x):   # x: 범주형 타입의 컬럼(Series)
#     le = LabelEncoder()
#     le.fit(x)
#     label = le.transform(x)
#     enc_classes[x.name] = le.classes_   # x.name: 컬럼명
#     return label

In [58]:
# card_df2 = card_df['card_tpbuz_cd'].apply(encoding_label)
# 오늘, 행정동에서, 어떤 업종에 방문한 연령대의 남성이 발생시킨 매출의 평균
grouped_df = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd", "sex", "age"]).agg({"amt": "mean", "cnt": "sum"}).reset_index()

grouped_df.head(5)

,ta_ymd,admi_cty_no,card_tpbuz_cd,sex,age,amt,cnt
0,20230101,41210510,D05,M,5,57576.0,2
1,20230101,41210510,D11,F,7,8273.0,4
2,20230101,41210510,F02,F,4,97650.0,2
3,20230101,41210510,F02,F,6,25789.0,2
4,20230101,41210510,F02,F,7,74074.0,2


In [59]:
grouped_df = grouped_df.drop(['ta_ymd'],axis=1)

In [60]:
train_df, test_df = train_test_split(grouped_df, test_size=0.2, random_state=42)

In [61]:
# train_size = int(len(card_df) * 0.8)
# train_df = card_df[:train_size]
# test_df = card_df[train_size:]

In [62]:
predictor = TabularPredictor(
    label='amt',  # 예측할 타겟 컬럼 설정
    eval_metric='mean_absolute_error'  # 평가 지표 설정
)

No path specified. Models will be saved in: "AutogluonModels\ag-20240711_033255"


In [ ]:
predictor.fit(train_data=train_df)

In [64]:
# 예측 수행
predictions = predictor.predict(test_df)

In [66]:
predictions.head()

1018321    3.377258e+05
2124564    2.413185e+04
2258131    1.342081e+06
2011751    2.614102e+05
1738195    7.445822e+04
Name: amt, dtype: float32

In [67]:
test_df.head()

,admi_cty_no,card_tpbuz_cd,sex,age,amt,cnt
1018321,41210590,S04,F,6,3.442950e+04,4
2124564,41210520,Q14,M,2,3.107500e+04,1
2258131,41210610,S04,M,7,3.930405e+06,66
2011751,41210550,F02,F,7,4.016005e+05,16
1738195,41210580,F03,M,4,6.081390e+06,2


In [68]:
predictor.leaderboard(test_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesMSE,-185856.722470,-185911.330726,mean_absolute_error,2.764904,0.123686,201.497280,2.764904,0.123686,201.497280,1,True,7
1,RandomForestMSE,-188120.379728,-187931.481231,mean_absolute_error,2.519405,0.124849,262.785172,2.519405,0.124849,262.785172,1,True,5
2,LightGBM,-190159.331894,-189084.193637,mean_absolute_error,3.645283,0.145639,19.794451,3.645283,0.145639,19.794451,1,True,4
3,LightGBMXT,-192731.062032,-192396.499009,mean_absolute_error,83.806180,3.146824,177.869431,83.806180,3.146824,177.869431,1,True,3
4,CatBoost,-216138.837812,-212478.484948,mean_absolute_error,0.667849,0.080087,1044.560808,0.667849,0.080087,1044.560808,1,True,6
5,KNeighborsUnif,-415725.153611,-446119.955703,mean_absolute_error,16.806362,1.236200,14.697853,16.806362,1.236200,14.697853,1,True,1
6,KNeighborsDist,-416299.692563,-446152.890681,mean_absolute_error,15.315404,0.645500,2.131970,15.315404,0.645500,2.131970,1,True,2


## 데이터 결합

### 월 별 데이터로 통합, 유동인구와 상권변화지표도 통합

In [119]:
# 해당일, 해당동의 해당 업종의 특정시간에 매출과 매출건수 총액
card_df2 = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd"]).agg({"amt": "sum", "cnt":"sum"}).reset_index()

In [120]:
card_df2.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,20230101,41210510,D05,57576,2
1,20230101,41210510,D11,16546,4
2,20230101,41210510,F02,313388,16
3,20230101,41210510,Q01,69266,1
4,20230101,41210510,Q15,78222,6


In [121]:
card_df2['ta_ymd'] = card_df2['ta_ymd'].astype(str) 

In [122]:
card_df2['ta_ymd'] = card_df2['ta_ymd'].str.slice(0, 6)

In [123]:
card_df2.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,202301,41210510,D05,57576,2
1,202301,41210510,D11,16546,4
2,202301,41210510,F02,313388,16
3,202301,41210510,Q01,69266,1
4,202301,41210510,Q15,78222,6


In [124]:
# 월별 행정구별, 업종별, amt 및 cnt 평균
grouped_df = card_df2.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd"]).agg({"amt": "sum", "cnt": "sum"}).reset_index()
grouped_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt
0,202301,41210510,D05,633562,28
1,202301,41210510,D11,171040,28
2,202301,41210510,F02,12060758,375
3,202301,41210510,Q01,1818710,9
4,202301,41210510,Q15,2167723,87


In [125]:
# join을 위한 데이터 통일
flowpop_df['ETL_M'] = flowpop_df['ETL_M'].str.replace('-', '')
# 총 유동인구 계산
flowpop_df['TOTAL_POPULATION'] = flowpop_df['M_CNT'] + flowpop_df['F_CNT']
# 해당 월에 해당 동에 해당 시간에 방문한 유동인구수 평균
flowpop_df_temp = flowpop_df.groupby(["ETL_M", "ADMI_CD", "ADMI_NM","TIME_CD"]).agg({"TOTAL_POPULATION": "mean"}).reset_index()
# 해당 월에 해당 동에 방문한 유동인구 평균 총합
flow_g_df = flowpop_df_temp.groupby(["ETL_M", "ADMI_CD", "ADMI_NM"]).agg({"TOTAL_POPULATION": "sum"}).reset_index()

In [126]:
flow_g_df.head()

,ETL_M,ADMI_CD,ADMI_NM,TOTAL_POPULATION
0,202301,41210510,광명1동,13850.98
1,202301,41210520,광명2동,383145.55
2,202301,41210540,광명3동,1505126.40
3,202301,41210550,광명4동,1221965.65
4,202301,41210560,광명5동,1344132.80


In [127]:
# 데이터 결합
merged_df = dd.merge(grouped_df, flow_g_df, how='inner', left_on=['ta_ymd', 'admi_cty_no'], right_on=['ETL_M', 'ADMI_CD'])


In [128]:
merged_df.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,ADMI_NM,TOTAL_POPULATION
0,202301,41210510,D05,633562,28,202301,41210510,광명1동,13850.98
1,202301,41210510,D11,171040,28,202301,41210510,광명1동,13850.98
2,202301,41210510,F02,12060758,375,202301,41210510,광명1동,13850.98
3,202301,41210510,Q01,1818710,9,202301,41210510,광명1동,13850.98
4,202301,41210510,Q15,2167723,87,202301,41210510,광명1동,13850.98


In [134]:
# 해당 행정동의 상권 지표를 결합
merged_df2 = dd.merge(merged_df, comm_df, how='inner', left_on=['ADMI_NM'], right_on=['행정동명'])

In [135]:
merged_df2.head()

,ta_ymd,admi_cty_no,card_tpbuz_cd,amt,cnt,ETL_M,ADMI_CD,ADMI_NM,TOTAL_POPULATION,행정시명,행정동명,운영점포평균영업기간,폐업점포평균영업기간,상권변동지표구분,상권변동지표구분명,기준분기
0,202301,41210510,D05,633562,28,202301,41210510,광명1동,13850.98,광명시,광명1동,188.5,358.0,HH,정체,2
1,202301,41210510,D11,171040,28,202301,41210510,광명1동,13850.98,광명시,광명1동,188.5,358.0,HH,정체,2
2,202301,41210510,F02,12060758,375,202301,41210510,광명1동,13850.98,광명시,광명1동,188.5,358.0,HH,정체,2
3,202301,41210510,Q01,1818710,9,202301,41210510,광명1동,13850.98,광명시,광명1동,188.5,358.0,HH,정체,2
4,202301,41210510,Q15,2167723,87,202301,41210510,광명1동,13850.98,광명시,광명1동,188.5,358.0,HH,정체,2


In [136]:
merged_df2.columns

Index(['ta_ymd', 'admi_cty_no', 'card_tpbuz_cd', 'amt', 'cnt', 'ETL_M',
       'ADMI_CD', 'ADMI_NM', 'TOTAL_POPULATION', '행정시명', '행정동명', '운영점포평균영업기간',
       '폐업점포평균영업기간', '상권변동지표구분', '상권변동지표구분명', '기준분기'],
      dtype='object')

In [137]:
# 필요없는 속성 제거
columns_to_drop = ['ta_ymd','ETL_M', 'ADMI_CD', 'ADMI_NM',  '행정시명', '행정동명',
                   '운영점포평균영업기간', '폐업점포평균영업기간', '상권변동지표구분명', '기준분기']
merged_df2 = merged_df2.drop(columns=columns_to_drop)

In [138]:
merged_df2.head()

,admi_cty_no,card_tpbuz_cd,amt,cnt,TOTAL_POPULATION,상권변동지표구분
0,41210510,D05,633562,28,13850.98,HH
1,41210510,D11,171040,28,13850.98,HH
2,41210510,F02,12060758,375,13850.98,HH
3,41210510,Q01,1818710,9,13850.98,HH
4,41210510,Q15,2167723,87,13850.98,HH


In [139]:
# 505027 rows
len(merged_df2)

505027

## One hot encoding, train-test split

In [142]:
from multiprocessing import Pool, cpu_count
cpu_count()

12

In [85]:
merged_df2 = merged_df2.compute()
merged_df2_encoded = pd.get_dummies(merged_df2, columns=['card_tpbuz_cd', '상권변동지표구분'])

# 변환된 DataFrame 확인
print(merged_df2_encoded.head())

# train/test 데이터 나누기 (80-20 비율)
train_df, test_df = train_test_split(merged_df2_encoded, test_size=0.2, random_state=42)

   admi_cty_no            amt       cnt  TOTAL_POPULATION  card_tpbuz_cd_D01  \
0     41210510   48735.538462  2.153846          13850.98              False   
1     41210510   12217.142857  2.000000          13850.98              False   
2     41210510   86768.043165  2.697842          13850.98              False   
3     41210510  259815.714286  1.285714          13850.98              False   
4     41210510   51612.452381  2.071429          13850.98              False   

   card_tpbuz_cd_D02  card_tpbuz_cd_D03  card_tpbuz_cd_D04  card_tpbuz_cd_D05  \
0              False              False              False               True   
1              False              False              False              False   
2              False              False              False              False   
3              False              False              False              False   
4              False              False              False              False   

   card_tpbuz_cd_D06  ...  card_

## 모델 학습1

In [90]:
# Autogluon TabularPredictor 사용
predictor = TabularPredictor(
    label='amt',  # 예측할 타겟 컬럼 설정
    eval_metric='mean_absolute_error'  # 평가 지표 설정 
)

# 모델 훈련
predictor.fit(
    train_data=train_df,
    presets='good_quality_faster_inference_only_refit',  # 가벼운 모델 설정
    time_limit=7200  # 학습 시간 2시간으로 제한 (7200초)
)

# 예측 수행
predictions = predictor.predict(test_df)

# 예측 결과 및 실제 값 비교
print("Predictions:\n", predictions)
print("Actual:\n", test_df['amt'])

No path specified. Models will be saved in: "AutogluonModels\ag-20240711_084122"
Preset alias specified: 'good_quality_faster_inference_only_refit' maps to 'good_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       19.14 GB / 31.69 GB (60.4%)
Disk Space Avail:   592.39 GB / 930.86 GB (63.6%)
Presets specified: ['good_quality_faster_inference_only_refit']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data

(_ray_fit pid=22216) [1000]	valid_set's l1: 294458
(_ray_fit pid=22216) [2000]	valid_set's l1: 274022 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=21984) [3000]	valid_set's l1: 268395 [repeated 8x across cluster]
(_ray_fit pid=21984) [4000]	valid_set's l1: 260611 [repeated 8x across cluster]
(_ray_fit pid=21984) [5000]	valid_set's l1: 254928 [repeated 8x across cluster]
(_ray_fit pid=21984) [6000]	valid_set's l1: 250245 [repeated 8x across cluster]
(_ray_fit pid=21984) [7000]	valid_set's l1: 246243 [repeated 8x across cluster]
(_ray_fit pid=21984) [8000]	valid_set's l1: 242651 [repeated 8x across cluster]
(_ray_fit pid=21984) [9000]	valid_set's l1: 239651 [repeated 8x across cluster]
(_ray_fit pid=21984) [10000]	valid_set's l1: 236989 [repeated 8x across cluster]


(_dystack pid=4876) 	-232048.7174	 = Validation score   (-mean_absolute_error)
(_dystack pid=4876) 	158.74s	 = Training   runtime
(_dystack pid=4876) 	1687.19s	 = Validation runtime
(_dystack pid=4876) Fitting model: LightGBM_BAG_L1 ... Training model for up to 817.14s of the 1413.79s of remaining time.
(_dystack pid=4876) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.48%)


(_ray_fit pid=24452) [1000]	valid_set's l1: 246309 [repeated 8x across cluster]
(_ray_fit pid=24452) [2000]	valid_set's l1: 218194 [repeated 8x across cluster]
(_ray_fit pid=23564) [3000]	valid_set's l1: 201739 [repeated 8x across cluster]
(_ray_fit pid=24452) [4000]	valid_set's l1: 194048 [repeated 8x across cluster]
(_ray_fit pid=20632) [5000]	valid_set's l1: 187046 [repeated 8x across cluster]
(_ray_fit pid=20632) [6000]	valid_set's l1: 181814 [repeated 8x across cluster]
(_ray_fit pid=20632) [7000]	valid_set's l1: 177476 [repeated 8x across cluster]
(_ray_fit pid=12276) [8000]	valid_set's l1: 175062 [repeated 8x across cluster]
(_ray_fit pid=18316) [9000]	valid_set's l1: 170580 [repeated 8x across cluster]
(_ray_fit pid=20632) [10000]	valid_set's l1: 168243 [repeated 8x across cluster]


(_dystack pid=4876) 	-167506.9702	 = Validation score   (-mean_absolute_error)
(_dystack pid=4876) 	128.59s	 = Training   runtime
(_dystack pid=4876) 	1203.31s	 = Validation runtime
(_dystack pid=4876) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 534.19s of the 1130.83s of remaining time.
(_dystack pid=4876) 	-310380.3175	 = Validation score   (-mean_absolute_error)
(_dystack pid=4876) 	181.47s	 = Training   runtime
(_dystack pid=4876) 	18.52s	 = Validation runtime
(_dystack pid=4876) Fitting model: CatBoost_BAG_L1 ... Training model for up to 333.79s of the 930.44s of remaining time.
(_dystack pid=4876) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.58%)
(_dystack pid=4876) 	-253869.9905	 = Validation score   (-mean_absolute_error)
(_dystack pid=4876) 	267.41s	 = Training   runtime
(_dystack pid=4876) 	1.21s	 = Validation runtime
(_dystack pid=4876) Fitting model: ExtraTreesMSE_BAG_L

(_ray_fit pid=20416) [1000]	valid_set's l1: 179220 [repeated 8x across cluster]
(_ray_fit pid=20416) [2000]	valid_set's l1: 173587 [repeated 8x across cluster]
(_ray_fit pid=20416) [3000]	valid_set's l1: 170690 [repeated 8x across cluster]
(_ray_fit pid=14104) [4000]	valid_set's l1: 163754 [repeated 8x across cluster]
(_ray_fit pid=14104) [5000]	valid_set's l1: 161814 [repeated 8x across cluster]
(_ray_fit pid=14104) [6000]	valid_set's l1: 160779 [repeated 8x across cluster]
(_ray_fit pid=14104) [7000]	valid_set's l1: 159547 [repeated 8x across cluster]
(_ray_fit pid=14104) [8000]	valid_set's l1: 158905 [repeated 8x across cluster]
(_ray_fit pid=14104) [9000]	valid_set's l1: 158232 [repeated 8x across cluster]
(_ray_fit pid=14104) [10000]	valid_set's l1: 157617 [repeated 8x across cluster]


(_dystack pid=4876) 	-161515.2333	 = Validation score   (-mean_absolute_error)
(_dystack pid=4876) 	162.48s	 = Training   runtime
(_dystack pid=4876) 	1231.98s	 = Validation runtime
(_dystack pid=4876) Fitting model: LightGBM_BAG_L2 ... Training model for up to 272.49s of the 272.47s of remaining time.
(_dystack pid=4876) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.67%)


(_ray_fit pid=22184) [1000]	valid_set's l1: 154280 [repeated 8x across cluster]
(_ray_fit pid=22184) [2000]	valid_set's l1: 149112 [repeated 8x across cluster]
(_ray_fit pid=22184) [3000]	valid_set's l1: 145684 [repeated 8x across cluster]
(_ray_fit pid=22184) [4000]	valid_set's l1: 143752 [repeated 8x across cluster]
(_ray_fit pid=22184) [5000]	valid_set's l1: 141839 [repeated 8x across cluster]
(_ray_fit pid=324) [6000]	valid_set's l1: 137404 [repeated 8x across cluster]
(_ray_fit pid=22184) [7000]	valid_set's l1: 138847 [repeated 8x across cluster]
(_ray_fit pid=324) [8000]	valid_set's l1: 134523 [repeated 8x across cluster]
(_ray_fit pid=18808) [9000]	valid_set's l1: 140720 [repeated 8x across cluster]
(_ray_fit pid=18808) [10000]	valid_set's l1: 139665 [repeated 8x across cluster]


(_dystack pid=4876) 	-132948.0347	 = Validation score   (-mean_absolute_error)
(_dystack pid=4876) 	148.82s	 = Training   runtime
(_dystack pid=4876) 	1126.79s	 = Validation runtime
(_dystack pid=4876) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -20.58s of remaining time.
(_dystack pid=4876) 	Ensemble Weights: {'LightGBM_BAG_L2': 0.96, 'LightGBM_BAG_L1': 0.04}
(_dystack pid=4876) 	-132916.6329	 = Validation score   (-mean_absolute_error)
(_dystack pid=4876) 	0.76s	 = Training   runtime
(_dystack pid=4876) 	0.01s	 = Validation runtime
(_dystack pid=4876) AutoGluon training complete, total runtime = 1812.32s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 11.2 rows/s (44892 batch size)
(_dystack pid=4876) Automatically performing refit_full as a post-fit operation (due to `.fit(..., refit_full=True)`
(_dystack pid=4876) Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
(_dystack pid

Predictions:
 16300     2.210050e+05
390581    4.589081e+05
458594    1.321966e+06
289786    3.900683e+04
106858    2.477182e+05
              ...     
73125     4.417539e+05
108234    1.597821e+06
164205    1.045358e+05
368943    5.241798e+05
203690    1.863706e+05
Name: amt, Length: 101006, dtype: float32
Actual:
 16300     2.348204e+05
390581    4.624939e+05
458594    1.332065e+06
289786    3.431855e+04
106858    2.908890e+05
              ...     
73125     4.036806e+05
108234    1.715494e+06
164205    1.203824e+05
368943    6.047262e+05
203690    2.435276e+05
Name: amt, Length: 101006, dtype: float64


In [93]:
y_pred = predictor.predict(test_df.drop(columns=['amt']))
y_pred.head()

16300     2.210050e+05
390581    4.589081e+05
458594    1.321966e+06
289786    3.900683e+04
106858    2.477182e+05
Name: amt, dtype: float32

In [95]:
predictor.evaluate(test_df, silent=True)

{'mean_absolute_error': -131853.82923148104,
 'root_mean_squared_error': -593713.411465162,
 'mean_squared_error': -352495614953.60065,
 'r2': 0.9065158026277896,
 'pearsonr': 0.9525155210651658,
 'median_absolute_error': -39506.59152770483}

In [96]:
predictor.leaderboard(test_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3_FULL,-131853.829231,NaN,mean_absolute_error,47.536544,NaN,1649.775785,0.003991,NaN,1.802965,3,True,32
1,LightGBM_BAG_L2_FULL,-134709.943866,NaN,mean_absolute_error,47.135546,NaN,1170.505009,11.236320,NaN,49.226270,2,True,28
2,RandomForestMSE_BAG_L2_FULL,-140768.033484,NaN,mean_absolute_error,36.296233,NaN,1598.746550,0.397007,19.663615,477.467811,2,True,29
3,CatBoost_BAG_L2_FULL,-143302.981888,NaN,mean_absolute_error,35.965566,NaN,1184.288002,0.066340,NaN,63.009263,2,True,30
4,LightGBMXT_BAG_L2_FULL,-151776.123884,NaN,mean_absolute_error,48.377680,NaN,1170.998634,12.478454,NaN,49.719894,2,True,27
5,ExtraTreesMSE_BAG_L2_FULL,-152207.608682,NaN,mean_absolute_error,36.055161,NaN,1200.253585,0.155935,6.137095,78.974845,2,True,31
6,LightGBM_BAG_L1_FULL,-160633.643526,NaN,mean_absolute_error,13.634603,NaN,28.811192,13.634603,NaN,28.811192,1,True,18
7,WeightedEnsemble_L2_FULL,-160681.208407,NaN,mean_absolute_error,15.058169,NaN,38.022117,0.010970,NaN,1.307649,2,True,26
8,XGBoost_BAG_L1_FULL,-199722.566837,NaN,mean_absolute_error,1.412596,NaN,7.903276,1.412596,NaN,7.903276,1,True,23
9,LightGBMLarge_BAG_L1_FULL,-204856.464302,NaN,mean_absolute_error,1.120790,NaN,5.435503,1.120790,NaN,5.435503,1,True,25
